In [4]:
from qiskit import __version__
print(__version__)

1.4.2


In [302]:
from qiskit import QuantumCircuit, ClassicalRegister
from qiskit.quantum_info import Statevector, state_fidelity, partial_trace
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from qiskit import transpile 
import numpy as np
from qiskit_aer.noise import NoiseModel, depolarizing_error, ReadoutError 
from qiskit.circuit.controlflow import IfElseOp
import matplotlib.pyplot as plt

In [410]:
import importlib
import steane_ec_decoder
importlib.reload(steane_ec_decoder)
from steane_ec_decoder import lookup

# Logical Error Rate for Encoding Circuit

# Stabilizer Extraction Circuits

In [396]:
def flag1(qc: QuantumCircuit, first_qubit: int, c1: ClassicalRegister):
    # Setting first ancilla to |+>
    qc.h(first_qubit+7)
    
    qc.cx(first_qubit+7, first_qubit+3)
    qc.cx(first_qubit+2, first_qubit+9)
    qc.cx(first_qubit+5, first_qubit+8)
    qc.cx(first_qubit+7, first_qubit+8)
    qc.cx(first_qubit+7, first_qubit)
    qc.cx(first_qubit+3, first_qubit+9)
    qc.cx(first_qubit+4, first_qubit+8)
    qc.cx(first_qubit+7, first_qubit+1)
    qc.cx(first_qubit+6, first_qubit+9)
    qc.cx(first_qubit+2, first_qubit+8)
    qc.cx(first_qubit+7, first_qubit+9)
    qc.cx(first_qubit+7, first_qubit+2)
    qc.cx(first_qubit+5, first_qubit+9)
    qc.cx(first_qubit+1, first_qubit+8)
    
    # Measure first ancilla in X-basis
    qc.h(first_qubit+7)
    
    qc.measure([first_qubit+7, first_qubit+8, first_qubit+9], c1)

In [397]:
def unflag(qc: QuantumCircuit, first_qubit: int, c2: ClassicalRegister):
    qc.h(first_qubit+7)
    qc.h(first_qubit+11)
    qc.h(first_qubit+12)
    
    qc.cx(first_qubit+7, first_qubit+3)
    qc.cx(first_qubit+2, first_qubit+9)
    qc.cx(first_qubit+5, first_qubit+8)
    qc.cx(first_qubit+7, first_qubit)
    qc.cx(first_qubit+3, first_qubit+9)
    qc.cx(first_qubit+4, first_qubit+8)
    qc.cx(first_qubit+7, first_qubit+1)
    qc.cx(first_qubit+6, first_qubit+9)
    qc.cx(first_qubit+2, first_qubit+8)
    qc.cx(first_qubit+7, first_qubit+2)
    qc.cx(first_qubit+5, first_qubit+9)
    qc.cx(first_qubit+1, first_qubit+8)
    
    qc.cx(first_qubit+3, first_qubit+10)
    qc.cx(first_qubit+12, first_qubit+2)
    qc.cx(first_qubit+11, first_qubit+5)
    qc.cx(first_qubit, first_qubit+10)
    qc.cx(first_qubit+12, first_qubit+3)
    qc.cx(first_qubit+11, first_qubit+4)
    qc.cx(first_qubit+1, first_qubit+10)
    qc.cx(first_qubit+12, first_qubit+6)
    qc.cx(first_qubit+11, first_qubit+2)
    qc.cx(first_qubit+2, first_qubit+10)
    qc.cx(first_qubit+12, first_qubit+5)
    qc.cx(first_qubit+11, first_qubit+1)
    
    
    qc.h(first_qubit+7)
    qc.h(first_qubit+11)
    qc.h(first_qubit+12)
    
    qc.measure([first_qubit+7, first_qubit+8, first_qubit+9, first_qubit+10, first_qubit+11, first_qubit+12], c2)


In [398]:
def flag2(qc: QuantumCircuit, first_qubit: int, c3: ClassicalRegister):
    # Setting last two ancillas to |+>
    qc.h(first_qubit+8)
    qc.h(first_qubit+9)
    
    qc.cx(first_qubit+3, first_qubit+7)
    qc.cx(first_qubit+9, first_qubit+2)
    qc.cx(first_qubit+8, first_qubit+5)
    qc.cx(first_qubit+8, first_qubit+7)
    qc.cx(first_qubit, first_qubit+7)
    qc.cx(first_qubit+9, first_qubit+3)
    qc.cx(first_qubit+8, first_qubit+4)
    qc.cx(first_qubit+1, first_qubit+7)
    qc.cx(first_qubit+9, first_qubit+6)
    qc.cx(first_qubit+8, first_qubit+2)
    qc.cx(first_qubit+9, first_qubit+7)
    qc.cx(first_qubit+2, first_qubit+7)
    qc.cx(first_qubit+9, first_qubit+5)
    qc.cx(first_qubit+8, first_qubit+1)
    
    # Measrue last two ancillas in the X-basis
    qc.h(first_qubit+8)
    qc.h(first_qubit+9)
    
    qc.measure([first_qubit+7, first_qubit+8, first_qubit+9], c3)

# Encoding + 1 Round of EC

In [399]:
noise_model = NoiseModel()

noise_model.add_all_qubit_quantum_error(depolarizing_error(0.005,1), ['id'])
noise_model.add_all_qubit_quantum_error(depolarizing_error(0.005,1), ['h'])
noise_model.add_all_qubit_quantum_error(depolarizing_error(0.05,2), ['cx'])

# 5% chance of flipping 0 <-> 1
readout_err = ReadoutError([[0.995, 0.005],  # P(measured 0 | actual 0), P(1 | 0)
                            [0.005, 0.995]]) # P(0 | 1), P(1 | 1)

# Apply to all qubits being measured
noise_model.add_readout_error(readout_err, [7])
noise_model.add_readout_error(readout_err, [8])
noise_model.add_readout_error(readout_err, [9])
noise_model.add_readout_error(readout_err, [10])
noise_model.add_readout_error(readout_err, [11])
noise_model.add_readout_error(readout_err, [12])

In [425]:
qc = QuantumCircuit(13)
# Classical Register for flag1 measurements
c1 = ClassicalRegister(3, "c1")
# Classical Register for unflag1 measurements 
c2 = ClassicalRegister(6, "c2")
# Classical Register for flag2 measurements
c3 = ClassicalRegister(3, "c3")
qc.add_register(c1, c2, c3)

# Encoding data qubits
theta = np.arctan(np.sqrt((np.sqrt(5) - 1) / 2))
amp_0 = np.cos(theta/2)
amp_1 = np.sin(theta/2)
qc.initialize([amp_0, amp_1], 0)
for i in range(7):
    qc.id(i)
for i in range(4, 7):
    qc.h(i)
qc.cx(0, 1)
qc.cx(0, 2)
qc.cx(6, 0)
qc.cx(6, 1)
qc.cx(6, 3)
qc.cx(5, 0)
qc.cx(5, 2)
qc.cx(5, 3)
qc.cx(4, 1)
qc.cx(4, 2)
qc.cx(4, 3)

# Measuring flag1 circuit
flag1(qc, 0, c1)
qc.reset([7,8,9])

# body is what's appended if flag1 measures 0,0,0
body = QuantumCircuit(13)
body.add_register(c1, c2, c3)
flag2(body, 0, c3)
body.reset([7,8,9])

# if flag1 and flag2 both measure 0,0,0 do nothing
do_nothing = QuantumCircuit(13)
do_nothing.add_register(c1, c2, c3)

# if flag1 measures 0,0,0 and flag2 doesn't measure 0,0,0 then do unflag
do_unflag = QuantumCircuit(13)
do_unflag.add_register(c1, c2, c3)
unflag(do_unflag, 0, c2)

nested_if = IfElseOp((c3, 0), true_body=do_nothing, false_body=do_unflag)
body.append(nested_if, list(range(13)), c1[:] + c2[:] + c3[:]) 

# if flag1 isn't 0,0,0 then do unflag
else_body = QuantumCircuit(13)
else_body.add_register(c1, c2, c3)
unflag(else_body, 0, c2)

top_if = IfElseOp((c1, 0), true_body=body, false_body=else_body)
qc.append(top_if, list(range(13)), c1[:] + c2[:] + c3[:])


backend = AerSimulator(noise_model=noise_model, method="statevector")
transpiled = transpile(qc, backend, optimization_level=0)
job = backend.run(transpiled, shots=1, memory=True)
result = job.result()
memory = result.get_memory()[0]

correction = lookup(memory)

In [426]:
print(memory)
split = memory.split()
print(split)

syndrome_flag1 = [int(b) for b in split[2]][::-1] 
syndrome_unflag = [int(b) for b in split[1]][::-1]
syndrome_flag2 = [int(b) for b in split[0]][::-1]

print(syndrome_flag1)
print(syndrome_unflag)
print(syndrome_flag2)

print(correction)

000 011100 101
['000', '011100', '101']
[1, 0, 1]
[0, 0, 1, 1, 1, 0]
[0, 0, 0]
('IIIIZZZ', 'IIIIIII')
